In [ ]:
import streamlit as st
import pandas as pd
import os
import pickle
from PIL import Image

# Constants
FEMALE_GENDER = 1
MALE_GENDER = 2

class BodyClassifierApp:
    def __init__(self):
        self.rf_model = self.load_rf_model()
        self.recommendation_images = {
            "Female": {
                "APPLE": {
                    "Skirt": ["images/Women/a line skirt .png", "images/Women/wrap skirt .png", "images/Women/handkerchief skirt .png", "images/Women/flip skirt .png", "images/Women/draped skirt .png"],
                    "Jumpsuits": ["images/Women/belted jumpsuit .png", "images/Women/wide leg jumpsuit .png", "images/Women/utility jumpsuit .png", "images/Women/wrap jumpsuit .png", "images/Women/empire jumpsuit .png"],
                    "Pants": ["images/Women/harem pants .png", "images/Women/bootcut pants.png", "images/Women/Palazzo pants .png", "images/Women/pegged pants.png", "images/Women/wideleg jeans pants.png"],
                    "Necklines": ["images/Women/y neckline .png", "images/Women/v neckline.png", "images/Women/sweetheart neckline .png", "images/Women/scoop neckline .png", "images/Women/off shoulder neckline .png"],
                    "Tops": ["images/Women/off shoulder top .png", "images/Women/peplum top .png", "images/Women/wrap top.png", "images/Women/empire top.png", "images/Women/hoodie - top.png"],
                    "Sleeves": ["images/Women/cap sleeve .png", "images/Women/Bell sleeve.png", "images/Women/dolman sleeve.png", "images/Women/flutter sleeve .png", "images/Women/off shoulder sleeve .png"],
                    "TRADITIONAL WEAR": ["images/Women/aline kurta.png", "images/Women/anarkali kurta.png", "images/Women/straight cut kurta.png", "images/Women/empire waist kurta.png", "images/Women/saree.png"]
                }
            },
            "Male": {}
        }


    def load_rf_model(self):
        try:
            with open('random_forest_model.pkl', 'rb') as file:
                model = pickle.load(file)
            return model
        except Exception as e:
            st.error(f"Failed to load the RandomForestClassifier model: {e}")
            return None

# Load the trained model
body_classifier = BodyClassifierApp()

# Load the dataset
try:
    dataset = pd.read_csv('dataset.csv')
except Exception as e:
    st.error(f"Failed to load dataset: {e}")

def classify(gender, age, measurements):
    try:
        data = pd.DataFrame(columns=['Gender', 'Age', 'Shoulder', 'Waist', 'Hips', 'Bust', 'Chest'])
        data.loc[0] = [FEMALE_GENDER if gender == "Female" else MALE_GENDER, age] + measurements
        
        if body_classifier.rf_model:
            body_type = body_classifier.rf_model.predict(data)[0]
            st.success(f"Predicted Body Type: {body_type}")
            provide_recommendations(body_type, gender)
        else:
            st.error("RandomForestClassifier Model not loaded.")
    except Exception as e:
        st.error(str(e))

def provide_recommendations(body_type, gender):
    try:
        filtered_data = dataset[(dataset['Gender'] == gender) & (dataset[body_type] == 1)]
        
        recommendations = {}
        for cloth_pattern in filtered_data['Cloth Pattern '].unique():
            pattern_data = filtered_data[filtered_data['Cloth Pattern '] == cloth_pattern]
            image_paths = pattern_data['Image Path'].tolist()
            recommendations[cloth_pattern] = image_paths
        
        display_recommendations(recommendations)
    except Exception as e:
        st.error(str(e))

def display_recommendations(recommendations):
    for cloth_pattern, image_paths in recommendations.items():
        st.subheader(f"Top 5 images for {cloth_pattern}:")
        for image_path in image_paths:
            st.image(image_path, caption=image_path, use_column_width=True)
            try:
                img = Image.open(full_image_path)
                col.image(img, use_column_width=True)
            except Exception as e:
                st.error(f"Error loading image: {e}")

st.title("Body Measurement Classifier")

# Gender selection
gender = st.selectbox("Gender:", ["Female", "Male"])

# Age input
age = st.number_input("Age:", min_value=0)

# Measurement inputs
measurement_labels = ["Shoulder", "Waist", "Hips", "Bust", "Chest"]
measurements = []
for label in measurement_labels:
    measurement = st.number_input(f"{label}:", min_value=0.0)
    measurements.append(measurement)

if st.button("Classify"):
    classify(gender, age, measurements)
